

```
# Trước khi chạy thì phải đổi môi trường runtime GPU ở mục Runtime -> change runtime type chọn GPU
# Nguồn tham khảo code: https://github.com/ageitgey/face_recognitionfbclid=IwAR09IR8tSv1huZs3Vy3jN7rZozDQWpJBuhyxyLb7SJQ8HAte8CKHfaoUu6U
```



Bước 1: kết nối colab với google drive

In [0]:
# import drive vào colab
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# chuyển đến thư mục drive
cd drive

/content/drive


In [0]:
# chuyển đến thư mục My drive
cd My Drive

/content/drive/My Drive


In [0]:
Bước 2: chuyển đường dẫn đến thư mục chứa code face recognition và cài đặc face_recognition module

In [0]:
# thay đổi đường dẫn đến thư mục chứa code face_recognition là computer_vision_technique
import os
os.chdir("/content/drive/My Drive/computer_vision_technique/")

In [0]:

# Cài đặt face_recognition
!pip3 install mtcnn pytesseract face_recognition

     |████████████████████████████████| 2.3MB 4.9MB/s 
     |████████████████████████████████| 174kB 48.7MB/s 
     |████████████████████████████████| 100.2MB 1.2MB/s 
  Stored in directory: /root/.cache/pip/wheels/85/81/65/6363fa5aafd7a155c896591e0c7c6e27b69642aa82b9cbf076
  Stored in directory: /root/.cache/pip/wheels/cd/4a/30/998e01b892300ba0ccce7b806b6e889794605a384dac81a49a
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built mtcnn pytesseract face-recognition-models


In [0]:
!ls
#!locate DATATRAIN

BT1.ipynb  DATATRAIN  Output		     Untitled	      Untitled1.ipynb
DataImage  mug.jpg    trained_knn_model.clf  Untitled0.ipynb


Bước 3: tổ chức dữ liệu dùng để huẩn luyên theo cấu trúc:
 <train_dir>/
         <person1>/
               <somename1>.jpeg
               <somename2>.jpeg
               ...
        <person2>/
               <somename1>.jpeg
               <somename2>.jpeg
              ... 
sau đó chạy code bên dưới và code sẽ thực hiện các bước như sau:
       1. huấn luyện KNN classifier 
       2. sử dụng KNN classifier đã huấn luyện để sử dụng cho việc dự đoán
 

In [0]:
import math
from sklearn import neighbors
import os
import os.path
import pickle
from PIL import Image, ImageDraw
import cv2
from pathlib import Path
import face_recognition
from face_recognition.face_recognition_cli import image_files_in_folder

# image formats to read
# Định dạng ảnh mà code có thể đọc
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

# define training function
# Định nghĩa hàm train
def train(train_dir, model_save_path=None, n_neighbors=None, knn_algo='ball_tree', verbose=False):
  # define training folders
  # Cách tổ chức thư mục ảnh cho từng người
    """
    Trains a k-nearest neighbors classifier for face recognition.
    :param train_dir: directory that contains a sub-directory for each known person, with its name.
     (View in source code to see train_dir example tree structure)
     Structure:
        <train_dir>/
        ├── <person1>/
        │   ├── <somename1>.jpeg
        │   ├── <somename2>.jpeg
        │   ├── ...
        ├── <person2>/
        │   ├── <somename1>.jpeg
        │   └── <somename2>.jpeg
        └── ...
    :param model_save_path: (optional) path to save model on disk
    :param n_neighbors: (optional) number of neighbors to weigh in classification. Chosen automatically if not specified
    :param knn_algo: (optional) underlying data structure to support knn.default is ball_tree
    :param verbose: verbosity of training
    :return: returns knn classifier that was trained on the given data.
    """
    X = []
    y = []
    # Loop through each person in the training set
    # Quét các thư mục chứa các ảnh trong tập FaceDB
    for class_dir in os.listdir(train_dir):
        if not os.path.isdir(os.path.join(train_dir, class_dir)):
            continue

        # Loop through each training image for the current person
        # Quét các file trong thư mục con và lấy các ra các khuôn mặt trả về face trong các tấm ảnh.
        for img_path in image_files_in_folder(os.path.join(train_dir, class_dir)):
            # load images in a folder
            # tải ảnh trong thử mục con lên
            image = face_recognition.load_image_file(img_path)
            # locate face areas
            # phát hiện vị trí các gương mặt
            face_bounding_boxes = face_recognition.face_locations(image)

            if len(face_bounding_boxes) != 1:
                # If there are no people (or too many people) in a training image, skip the image.
                if verbose:
                    print("Image {} not suitable for training: {}".format(img_path, "Didn't find a face" if len(face_bounding_boxes) < 1 else "Found more than one face"))
            else:
                # Add face encoding for current image to the training set
                # thêm đặc trưng face encoding cho ảnh hiện tại vào tập huấn luyện
                X.append(face_recognition.face_encodings(image, known_face_locations=face_bounding_boxes)[0])
                y.append(class_dir)

    # Determine how many neighbors to use for weighting in the KNN classifier
    # Chọn K người láng giềng gần nhất
    if n_neighbors is None:
        n_neighbors = int(round(math.sqrt(len(X))))
        if verbose:
            print("Chose n_neighbors automatically:", n_neighbors)

    # Create and train the KNN classifier
    # huấn luyện KNN classifier với k = n_neighbors 
    knn_clf = neighbors.KNeighborsClassifier(n_neighbors=n_neighbors, algorithm=knn_algo, weights='distance')
    knn_clf.fit(X, y)

    # Save the trained KNN classifier
    # Lưu file vào thư mục model_save_path
    if model_save_path is not None:
        with open(model_save_path, 'wb') as f:
            pickle.dump(knn_clf, f)

    return knn_clf
  
# define face recognition function -> input an image and return the result
# Định nghĩa hàm nhận dạng khuôn mặt => đưa vào 1 tấm ảnh và trả về kết quả.
def predict(X_img_path, knn_clf=None, model_path=None, distance_threshold=0.75):
    """
    Recognizes faces in given image using a trained KNN classifier
    :param X_img_path: path to image to be recognized
    :param knn_clf: (optional) a knn classifier object. if not specified, model_save_path must be specified.
    :param model_path: (optional) path to a pickled knn classifier. if not specified, model_save_path must be knn_clf.
    :param distance_threshold: (optional) distance threshold for face classification. the larger it is, the more chance
           of mis-classifying an unknown person as a known one.
    :return: a list of names and face locations for the recognized faces in the image: [(name, bounding box), ...].
        For faces of unrecognized persons, the name 'unknown' will be returned.
    """
    if not os.path.isfile(X_img_path) or os.path.splitext(X_img_path)[1][1:] not in ALLOWED_EXTENSIONS:
        raise Exception("Invalid image path: {}".format(X_img_path))

    if knn_clf is None and model_path is None:
        raise Exception("Must supply knn classifier either thourgh knn_clf or model_path")

    # Load a trained KNN model (if one was passed in)
    # tải model KNN
    if knn_clf is None:
        with open(model_path, 'rb') as f:
            knn_clf = pickle.load(f)

    # Load image file and find face locations
    # Lấy khuôn mặt từ ảnh test
    X_img = face_recognition.load_image_file(X_img_path)
    X_face_locations = face_recognition.face_locations(X_img)

    # If no faces are found in the image, return an empty result.
    if len(X_face_locations) == 0:
        return []

    # Find encodings for faces in the test iamge
    # Rút trích đặc trưng khuôn mặt
    faces_encodings = face_recognition.face_encodings(X_img, known_face_locations=X_face_locations)

    # Use the KNN model to find the best matches for the test face
    # sử dụng KNN để tìm các gương mặt được cho khớp nhất giữa các gương mặt thông qua faces_encodings
    closest_distances = knn_clf.kneighbors(faces_encodings, n_neighbors=1)
    are_matches = [closest_distances[0][i][0] <= distance_threshold for i in range(len(X_face_locations))]

    # Predict classes and remove classifications that aren't within the threshold
    # dự đoán các lớp và loại các phân lớp không đáp ứng threshold
    return [(pred, loc) if rec else ("unknown", loc) for pred, loc, rec in zip(knn_clf.predict(faces_encodings), X_face_locations, are_matches)]


def show_prediction_labels_on_image(img_path, predictions):
    """
    Shows the face recognition results visually.
    :param img_path: path to image to be recognized
    :param predictions: results of the predict function
    :return:
    """
    pil_image = Image.open(img_path).convert("RGB")
    draw = ImageDraw.Draw(pil_image)

    for name, (top, right, bottom, left) in predictions:
        # Draw a box around the face using the Pillow module
        draw.rectangle(((left, top), (right, bottom)), outline=(0, 0, 255))

        # There's a bug in Pillow where it blows up with non-UTF-8 text
        # when using the default bitmap font
        name = name.encode("UTF-8")

        # Draw a label with a name below the face
        # vẽ các nhãn của các rương mặt đã được nhận diện
        text_width, text_height = draw.textsize(name)
        text_width = text_width 
        text_height = text_height
        draw.rectangle(((left, bottom - text_height - 10), (right, bottom)), fill=(0, 0, 255), outline=(0, 0, 255))
        draw.text((left + 6, bottom - text_height - 8), name, fill=(255, 255, 255, 255))
        #draw.text((10,60), name, fill=(255, 255, 255, 255))
        #draw.text((18,60), "World", font=fnt, fill=(255,255,255,255))

    # Remove the drawing library from memory as per the Pillow docs
    del draw

    # Display the resulting image
    # lưu các ảnh kết quả vào thư mục output
    pil_image.show()
    pil_image.save("Output/" + Path(img_path).name)


if __name__ == "__main__":
    # STEP 1: Train the KNN classifier and save it to disk
    # bước 1 huấn luyện KNN classifier và lưu nó và ổ đĩa
    # Once the model is trained and saved, you can skip this step next time.
    # trường hợp mô hình được huấn luyền và lưu lại thì có thể bỏ qua bước này
    print("Training KNN classifier...")
    classifier = train("./FaceDB", model_save_path="trained_knn_model.clf", n_neighbors=5)
    print("Training complete!")

    # STEP 2: Using the trained classifier, make predictions for unknown images
    # bước 2 sử dụng bộ KNN classifier đã được huấn luyện để thực hiện dự đoán cho các ảnh test
    for image_file in os.listdir("DataImage"):
        full_file_path = os.path.join("DataImage", image_file)

        print("Looking for faces in {}".format(image_file))

        # Find all people in the image using a trained classifier model
        # tìm tất cả người có trong ảnh sử dụng bộ KNN classifier đã được huấn luyện
        # Note: You can pass in either a classifier file name or a classifier model instance
        predictions = predict(full_file_path, model_path="trained_knn_model.clf")

        # Print results on the console
        # xuất kết quả dự đoán ra màn hình với tọa độ left top bounding box
        for name, (top, right, bottom, left) in predictions:
            print("- Found {} at ({}, {})".format(name, left, top))

        # Display results overlaid on an image
        show_prediction_labels_on_image(os.path.join("DataImage", image_file), predictions)

Training KNN classifier...
Training complete!
Looking for faces in IMG_5328.png
- Found HUNGLM at (619, 578)
- Found MINHHA at (1748, 944)
- Found TRUONGTX at (778, 903)
- Found VULQ at (838, 504)
- Found DUYNN at (139, 758)
- Found TIENBDT at (478, 519)
- Found QUANVM at (902, 554)
- Found THAODNT at (1199, 495)
- Found DUYETLV at (554, 713)
- Found LOCTH at (1429, 669)
- Found TANTD at (915, 713)
Looking for faces in IMG_5329.png
- Found MINHHA at (643, 375)
- Found HUNGLM at (306, 378)
- Found VULQ at (1380, 445)
- Found FACE3 at (984, 737)
- Found TIENBDT at (1722, 354)
- Found QUANVM at (64, 477)
- Found TULG at (991, 305)
Looking for faces in IMG_5330.png
- Found HUNGLM at (760, 378)
- Found QUANVM at (626, 483)
- Found MINHHA at (1081, 365)
- Found DUYETLV at (38, 454)
- Found FACE1 at (1429, 469)
- Found QUANVM at (304, 861)
- Found TULG at (1419, 285)
- Found VULQ at (1799, 402)
Looking for faces in IMG_5331.png
- Found DUYETLV at (1164, 683)
- Found TIENBDT at (151, 629)
- Fo